In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import dicom
import pydicom
import os
import seaborn as sns
from operator import itemgetter
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde
import warnings
import plotly 
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

warnings.filterwarnings('ignore')
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np

In [2]:
train_df = pd.read_csv('../data/train_2016_v2.csv',index_col=False, parse_dates=['transactiondate'])
#properties_df = pd.read_csv('../data/properties_2016.csv')
mergelogerror_properties=pd.read_csv('../data/train_merged_class_2016.csv')  
mergelogerror_properties_plotclass=pd.read_csv('../data/train_merged_plotclass_2016.csv') 
mergelogerror_properties_plotclass.head()    
mergelogerror_properties_plotclass.describe()    

,Unnamed: 0,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,Unnamed: 0.1,logerror,plotclass
count,90275.000000,9.027500e+04,28781.000000,261.000000,43.000000,90275.000000,90275.000000,16.0,57364.000000,89093.000000,...,8.989500e+04,9.027400e+04,90275.0,9.027400e+04,90269.000000,1783.000000,8.967000e+04,90275.000000,90275.000000,89428.000000
mean,45137.000000,1.298466e+07,1.816372,7.229885,713.581395,2.279474,3.031869,4.0,5.565407,2.309216,...,1.800934e+05,4.576726e+05,2015.0,2.783353e+05,5983.975927,13.402692,6.049151e+13,45137.000000,0.011457,0.253847
std,26060.292113,2.504510e+06,2.974168,2.716196,437.434198,1.004271,1.156436,0.0,1.900602,0.976172,...,2.091299e+05,5.548844e+05,0.0,4.004955e+05,6838.876956,2.715966,2.046605e+11,26060.292113,0.161079,0.435213
min,0.000000,1.071174e+07,1.000000,2.000000,100.000000,0.000000,0.000000,4.0,1.000000,1.000000,...,1.000000e+02,2.200000e+01,2015.0,2.200000e+01,49.080000,6.000000,6.037101e+13,0.000000,-4.605000,0.000000
25%,22568.500000,1.155950e+07,1.000000,7.000000,407.500000,2.000000,2.000000,4.0,4.000000,2.000000,...,8.124500e+04,1.990232e+05,2015.0,8.222800e+04,2872.830000,13.000000,6.037320e+13,22568.500000,-0.025300,0.000000
50%,45137.000000,1.254734e+07,1.000000,7.000000,616.000000,2.000000,3.000000,4.0,7.000000,2.000000,...,1.320000e+05,3.428720e+05,2015.0,1.929700e+05,4542.750000,14.000000,6.037615e+13,45137.000000,0.006000,0.000000
75%,67705.500000,1.422755e+07,1.000000,7.000000,872.000000,3.000000,4.000000,4.0,7.000000,3.000000,...,2.105345e+05,5.405890e+05,2015.0,3.454195e+05,6901.090000,15.000000,6.059042e+13,67705.500000,0.039200,1.000000
max,90274.000000,1.629608e+08,13.000000,21.000000,1555.000000,20.000000,16.000000,4.0,12.000000,20.000000,...,9.948100e+06,2.775000e+07,2015.0,2.450000e+07,321936.090000,99.000000,6.111009e+13,90274.000000,4.737000,1.000000


In [3]:
duplicatedtrain=train_df[train_df.duplicated(['parcelid'], keep=False)]
duplicatedtrain.head()

,parcelid,logerror,transactiondate
496,13850164,-0.1567,2016-01-05
497,13850164,-0.0460,2016-06-29
781,14677191,-0.3682,2016-01-06
782,14677191,-0.0845,2016-09-12
813,11005771,-0.0131,2016-01-06


In [4]:
print(duplicatedtrain['parcelid'].value_counts().describe())
print((duplicatedtrain['parcelid'].value_counts()==3).sum())
print((duplicatedtrain['parcelid'].value_counts()==2).sum())
print(duplicatedtrain['parcelid'].value_counts().head())
doubleduplicated=duplicatedtrain[~(duplicatedtrain['parcelid']==11842707)]
doubleduplicated1=pd.DataFrame(doubleduplicated['logerror'])
errorchange=[]
print (doubleduplicated1.head()) 

count    124.000000
mean       2.008065
std        0.089803
min        2.000000
25%        2.000000
50%        2.000000
75%        2.000000
max        3.000000
Name: parcelid, dtype: float64
1
123
11842707    3
13991964    2
12545874    2
11503158    2
14444102    2
Name: parcelid, dtype: int64
     logerror
496   -0.1567
497   -0.0460
781   -0.3682
782   -0.0845
813   -0.0131


In [5]:
from sklearn import preprocessing
from pandas.tools.plotting import scatter_matrix
#mergelogerror_properties = pd.merge(properties_df,train_df, on='parcelid')
#%matplotlib inline 
locationproperties=mergelogerror_properties[['longitude','latitude','regionidzip','regionidcity','regionidneighborhood','class']].sample(5000)
locationproperties['longitude']=locationproperties.longitude/1e6
locationproperties['latitude']=locationproperties.latitude/1e6
locationproperties1=locationproperties.dropna()
#sns.pairplot(locationproperties1,diag_kind="kde",hue="class",markers='+',plot_kws=dict(s=1,edgecolor="g", linewidth=1),diag_kws=dict(shade=True))

In [ ]:
from plotly.graph_objs import Scatter, Marker, Layout, XAxis, YAxis
import plotly.plotly as py
sample1=mergelogerror_properties.sample(5000)
lowerror1 = sample1[(sample1['class'] =='lowerror')]
higherror1 = sample1[(sample1['class']=='higherror')]

trace_comp0 = go.Scatter(
    y=lowerror1.latitude/1e6,
    x=lowerror1.longitude/1e6,
    mode='markers',
    marker=Marker(size=lowerror1.logerror, sizemode='area', sizeref=0.01,color='navy'),
    name='lowerror',
    text=lowerror1.logerror,
    )

trace_comp1 = go.Scatter(
    y=higherror1.latitude/1e6,
    x=higherror1.longitude/1e6,
    mode='markers',
    marker=Marker(size=higherror1.logerror, sizemode='area', sizeref=0.01,color='red'),
    name='higherror',
    text=higherror1.logerror,
        )

data_comp = [trace_comp0, trace_comp1]
layout_comp = go.Layout(
    title='Location VS logerror',
    hovermode='closest',
    xaxis=dict(
        title='longitude)',
        ticklen=5,
        zeroline=False,
        gridwidth=2,
    ),
    yaxis=dict(
        title='latitude',
        ticklen=5,
        gridwidth=2,
    ),
)
fig_comp = go.Figure(data=data_comp, layout=layout_comp)
#py.iplot(fig_comp, filename='location vs logerror')

In [6]:
structioncondition=mergelogerror_properties[['parcelid','bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt','buildingqualitytypeid','heatingorsystemtypeid','logerror','class']]
structioncondition['lowerror']=[0 if i!='lowerror' else 1 for i in structioncondition['class']]

#print(structioncondition['class'])
errorgroup = pd.crosstab([structioncondition['heatingorsystemtypeid'],
                    structioncondition['buildingqualitytypeid']], 
                    structioncondition['class'])
print(errorgroup)
#errorgroup.plot(kind='bar', stacked=True)
error_rate = errorgroup.div(errorgroup.sum(1).astype(float),
                             axis=0) # normalize the value

# print survival_rate
error_rate.plot(kind='barh', 
                   stacked=True)
plt.title('lowerror and high error percentage by heating system type and building quality type')
#plt.show()

class                                        higherror  lowerror
heatingorsystemtypeid buildingqualitytypeid                     
2.0                   1.0                          771      1738
                      4.0                         5052     17271
                      6.0                            0         1
                      7.0                         2725      8362
                      8.0                            2         1
                      10.0                         545       857
                      11.0                           0         1
                      12.0                          50        65
7.0                   1.0                           18        38
                      4.0                          257       601
                      7.0                         4583      9829
                      10.0                           5         6
20.0                  1.0                            1         3
                      4.0

Text(0.5,1,'lowerror and high error percentage by heating system type and building quality type')

In [7]:
structioncondition=mergelogerror_properties[['parcelid','bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt','buildingqualitytypeid','heatingorsystemtypeid','logerror','class']]
structioncondition['lowerror']=[0 if i!='lowerror' else 1 for i in structioncondition['class']]
structionconditionnumber=structioncondition[['buildingqualitytypeid','bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt']]
structionconditionnumber.describe()

,buildingqualitytypeid,bedroomcnt,bathroomcnt,roomcnt,fullbathcnt,calculatedbathnbr,unitcnt
count,57364.000000,90275.000000,90275.000000,90275.000000,89093.000000,89093.000000,58353.000000
mean,5.565407,3.031869,2.279474,1.478516,2.241231,2.309216,1.110414
std,1.900602,1.156436,1.004271,2.819627,0.963142,0.976172,0.797235
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,4.000000,2.000000,2.000000,0.000000,2.000000,2.000000,1.000000
50%,7.000000,3.000000,2.000000,0.000000,2.000000,2.000000,1.000000
75%,7.000000,4.000000,3.000000,0.000000,3.000000,3.000000,1.000000
max,12.000000,16.000000,20.000000,18.000000,20.000000,20.000000,143.000000


In [8]:
timeproperties=mergelogerror_properties[['yearbuilt','assessmentyear','logerror','class']]
timeproperties.describe()

,yearbuilt,assessmentyear,logerror
count,89519.000000,90275.0,90275.000000
mean,1968.532870,2015.0,0.011457
std,23.763475,0.0,0.161079
min,1885.000000,2015.0,-4.605000
25%,1953.000000,2015.0,-0.025300
50%,1970.000000,2015.0,0.006000
75%,1987.000000,2015.0,0.039200
max,2015.000000,2015.0,4.737000


In [9]:
logerrortimemean=timeproperties[['logerror','yearbuilt']].groupby('yearbuilt'). mean()
logerrortimemean.describe()

,logerror
count,130.000000
mean,0.012458
std,0.041136
min,-0.115633
25%,0.005236
50%,0.011144
75%,0.018542
max,0.405500


In [10]:
timeproperties=timeproperties[['yearbuilt','logerror','class']]
timeproperties.head()

,yearbuilt,logerror,class
0,1986.0,0.0953,higherror
1,1990.0,0.0198,lowerror
2,1956.0,0.0060,lowerror
3,1965.0,-0.0566,lowerror
4,1984.0,0.0573,lowerror


In [11]:
train_df['month']=train_df['transactiondate'].dt.month
train_df['month'].head()

0    1
1    1
2    1
3    1
4    1
Name: month, dtype: int64

In [12]:
trainlogerrormean=train_df[['logerror','month']].groupby('month'). mean()
trainlogerrormean.describe()

,logerror
count,12.000000
mean,0.012668
std,0.004299
min,0.006605
25%,0.009193
50%,0.013099
75%,0.016125
max,0.019122


In [13]:
abstrain_2016=train_df
abstrain_2016['logerror']=abs(train_df['logerror'])
abstrain_2016mean=abstrain_2016[['logerror','month']].groupby('month'). mean()
abstrain_2016mean.head()

,logerror
month,
1,0.072695
2,0.077434
3,0.072044
4,0.069972
5,0.066241


In [14]:
#mergelogerror_properties['month']=mergelogerror_properties['transactiondate'].dt.month
mergelogerror_properties['transactiondate'] = pd.to_datetime(mergelogerror_properties['transactiondate'], errors='coerce')
mergelogerror_properties['month']=mergelogerror_properties['transactiondate'].dt.month
#mergelogerror_properties['transactiondate'][0]
mergelogerror_properties.head()

,Unnamed: 0,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,Unnamed: 0.1,logerror,transactiondate,class,month
0,0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,...,76724.0,2015.06,NaN,NaN,6.111002e+13,5557,0.0953,2016-01-27,higherror,1
1,1,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,95870.0,2581.30,NaN,NaN,6.111002e+13,20707,0.0198,2016-03-30,lowerror,3
2,2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,14234.0,591.64,NaN,NaN,6.111001e+13,39717,0.0060,2016-05-27,lowerror,5
3,3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,...,17305.0,682.78,NaN,NaN,6.111001e+13,42866,-0.0566,2016-06-07,lowerror,6
4,4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,...,277000.0,5886.92,NaN,NaN,6.111001e+13,63903,0.0573,2016-08-08,lowerror,8


In [15]:
areapropertyland=mergelogerror_properties[['parcelid','calculatedfinishedsquarefeet','finishedsquarefeet12','lotsizesquarefeet','taxamount','taxvaluedollarcnt','structuretaxvaluedollarcnt','propertycountylandusecode','propertyzoningdesc','landtaxvaluedollarcnt','logerror','class']]
areapropertyland.describe()

,parcelid,calculatedfinishedsquarefeet,finishedsquarefeet12,lotsizesquarefeet,taxamount,taxvaluedollarcnt,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,logerror
count,9.027500e+04,89614.000000,85596.000000,8.012500e+04,90269.000000,9.027400e+04,8.989500e+04,9.027400e+04,90275.000000
mean,1.298466e+07,1773.185987,1745.454531,2.911016e+04,5983.975927,4.576726e+05,1.800934e+05,2.783353e+05,0.011457
std,2.504510e+06,928.162393,909.941166,1.217213e+05,6838.876956,5.548844e+05,2.091299e+05,4.004955e+05,0.161079
min,1.071174e+07,2.000000,2.000000,1.670000e+02,49.080000,2.200000e+01,1.000000e+02,2.200000e+01,-4.605000
25%,1.155950e+07,1184.000000,1172.000000,5.703000e+03,2872.830000,1.990232e+05,8.124500e+04,8.222800e+04,-0.025300
50%,1.254734e+07,1540.000000,1518.000000,7.200000e+03,4542.750000,3.428720e+05,1.320000e+05,1.929700e+05,0.006000
75%,1.422755e+07,2095.000000,2056.000000,1.168600e+04,6901.090000,5.405890e+05,2.105345e+05,3.454195e+05,0.039200
max,1.629608e+08,22741.000000,20013.000000,6.971010e+06,321936.090000,2.775000e+07,9.948100e+06,2.450000e+07,4.737000


In [16]:
from sklearn.decomposition import PCA
from sklearn.decomposition.pca import PCA
from sklearn import preprocessing
df=mergelogerror_properties[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet','logerror']].copy()
df=df.dropna()
df_scaled = preprocessing.scale(df[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet']]

print(df_scaled.head())
y = df.logerror
target_names='logerror'


 # fit data and then transform it
for k in range(1,6):
    pca=PCA(n_components=k)
    X_pca = pca.fit(df_scaled).transform(df_scaled)
    pca.fit(df_scaled)
    print('explained_variance_ratio:',pca.explained_variance_ratio_)
    print('pca:',pca.components_)

   longitude  latitude  fullbathcnt  calculatedbathnbr  buildingqualitytypeid  \
0  -1.776171  0.337964     0.795904           0.795904              -0.794267   
1  -1.728581  0.355357    -0.230965          -0.230965               0.782779   
2  -1.596012  0.356837    -0.230965          -0.230965               0.782779   
3  -1.730916  0.279418     0.795904           0.795904              -0.794267   
4  -1.895581  0.226098     4.903379           4.903379               2.359825   

   heatingorsystemtypeid  yearbuilt  taxamount  lotsizesquarefeet  
0              -0.639121   0.777665   0.189054           0.222133  
1              -0.639121  -0.166739  -0.070753          -0.253716  
2              -0.639121  -0.209667  -0.118929          -0.223060  
3              -0.639121  -0.037957   0.234378          -0.180141  
4              -0.639121   1.421576   2.820711          -0.152305  
explained_variance_ratio: [ 0.36227961]
pca: [[-0.15515591  0.18104453  0.48952015  0.48952015 -0.3046250

In [17]:
#import plotly.plotly as py
from plotly.graph_objs import *
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
#import plotly.tools as tls
from sklearn.preprocessing import StandardScaler
X=df_scaled
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
#print('Covariance matrix \n%s' %cov_mat)
#print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat2 = np.corrcoef(X.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
u,s,v = np.linalg.svd(X_std.T)
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
# Make a list of (eigenvalue, eigenvector) tuples

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('the cum')
print(cum_var_exp)


#fig = Figure(data=data1, layout=layout)
#py.iplot(Figure(data=data1, layout=layout))

Eigenvalues in descending order:
3.26051649259
1.44200041538
1.04122458673
0.955772420598
0.817062635364
0.528605440692
0.519362513599
0.435455495052
1.80652938808e-17
the cum
[  36.22796103   52.25018787   63.81934994   74.4390435    83.51751723
   89.39091101   95.16160561  100.          100.        ]


In [18]:
trace1 = Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data1 = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Number of rooms variables: Explained variance by different principal components ')

In [19]:
X=df_scaled[['fullbathcnt','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']]
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
#print('Covariance matrix \n%s' %cov_mat)
#print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat2 = np.corrcoef(X.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
#u,s,v = np.linalg.svd(X_std.T)


In [21]:
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
# Make a list of (eigenvalue, eigenvector) tuples

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('the cum')
print(cum_var_exp)
trace1 = Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data1 = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Number of rooms variables: Explained variance by different principal components ')

#fig = Figure(data=data1, layout=layout)
#py.iplot(Figure(data=data1, layout=layout))

Eigenvalues in descending order:
1.98521564449
1.14556380487
0.908377143163
0.56158989267
0.399253514806
the cum
[  39.70431289   62.61558899   80.78313185   92.0149297   100.        ]


In [23]:
X=df_scaled[['fullbathcnt','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']]
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
#print('Covariance matrix \n%s' %cov_mat)
#print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat2 = np.corrcoef(X.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
#u,s,v = np.linalg.svd(X_std.T)
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
# Make a list of (eigenvalue, eigenvector) tuples

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('the cum')
print(cum_var_exp)
trace1 = Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data1 = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Number of rooms variables: Explained variance by different principal components ')

#fig = Figure(data=data1, layout=layout)
#py.iplot(Figure(data=data1, layout=layout))

Eigenvalues in descending order:
1.98521564449
1.14556380487
0.908377143163
0.56158989267
0.399253514806
the cum
[  39.70431289   62.61558899   80.78313185   92.0149297   100.        ]


In [26]:

mergelogerror_properties_plotclass['yearbuilt'] = pd.to_datetime(mergelogerror_properties_plotclass['yearbuilt'], errors='coerce')
type(mergelogerror_properties_plotclass['yearbuilt'][0])

pandas._libs.tslib.Timestamp

In [32]:
#meansd=abs(-0.16107794320832886/2+0.011457219606756682)
#train_df['plotclass'] = pd.cut(abs(train_df.logerror),[0,meansd,1e6],3,
#                                 labels=['0','1']) # this creates a new variable
#mergelogerror_properties_plotclass = pd.merge(properties_df,train_df, on='parcelid')
df=mergelogerror_properties_plotclass[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet','logerror','plotclass']].sample(1000).copy()
df=df.dropna()
df_scaled = preprocessing.scale(df[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']]
#print(df.plotclass.describe())
y=df.plotclass

target_names='logerror'
 # fit data and then transform it
for k in range(1,6):
    pca=PCA(n_components=k)
    X_pca = pca.fit(df_scaled).transform(df_scaled)
    pca.fit(df_scaled)
    #print('explained_variance_ratio:',pca.explained_variance_ratio_)
    #print('pca:',pca.components_)
    from pandas.plotting import scatter_matrix
def get_feature_names_from_weights(weights, names):
    tmp_array = []
    for comp in weights:
        tmp_string = ''
        for fidx,f in enumerate(names):
            if fidx>0 and comp[fidx]>=0:
                tmp_string+='+'
            tmp_string += '%.2f*%s ' % (comp[fidx],f[:-5])
        tmp_array.append(tmp_string)
    return tmp_array
  
pca_weight_strings = get_feature_names_from_weights(pca.components_, df_scaled.columns) 
df_pca = pd.DataFrame(X_pca,columns=[pca_weight_strings])
y=preprocessing.scale(y)
#ax = scatter_matrix(df_pca, pca_weight_strings[0], pca_weight_strings[1], c=y,s=(y+2)*10)


In [31]:
pca_weight_strings[0]

'-0.17*long +0.14*lat +0.53*fullba +0.53*calculatedba -0.36*buildingqualityt -0.40*heatingorsystemt +0.32*taxa +0.06*lotsizesquar '

In [34]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition.pca import PCA
from sklearn import preprocessing
df=mergelogerror_properties_plotclass[['bedroomcnt','bathroomcnt','fullbathcnt','calculatedbathnbr','unitcnt','plotclass']].copy()
df=df.dropna()
df_scaled = preprocessing.scale(df[['bedroomcnt','bathroomcnt','fullbathcnt','calculatedbathnbr','unitcnt']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['bedroomcnt','bathroomcnt','fullbathcnt','calculatedbathnbr','unitcnt']]

print(df_scaled.head())
y = df.plotclass
target_names='plotclass'

for k in range(1,3):
    pca=PCA(n_components=k)
    X_pca = pca.fit(df_scaled).transform(df_scaled)
    pca.fit(df_scaled)
    print('explained_variance_ratio:',pca.explained_variance_ratio_)
    print('pca:',pca.components_)

from pandas.tools.plotting import scatter_matrix
def get_feature_names_from_weights(weights, names):
    tmp_array = []
    for comp in weights:
        tmp_string = ''
        for fidx,f in enumerate(names):
            if fidx>0 and comp[fidx]>=0:
                tmp_string+='+'
            tmp_string += '%.2f*%s ' % (comp[fidx],f[:-5])
        tmp_array.append(tmp_string)
    return tmp_array
  
pca_weight_strings = get_feature_names_from_weights(pca.components_, df_scaled.columns) 
df_pca = pd.DataFrame(X_pca,columns=[pca_weight_strings])
y=preprocessing.scale(y)
#ax = scatter_matrix(df_pca, pca_weight_strings[0], pca_weight_strings[1], c=y,s=(y+2)*10)

   bedroomcnt  bathroomcnt  fullbathcnt  calculatedbathnbr   unitcnt
0   -0.042264     0.692169     0.692191           0.692169 -0.126626
1   -0.042264    -0.270162    -0.270147          -0.270162 -0.126626
2    0.814388    -0.270162    -0.270147          -0.270162 -0.126626
3    0.814388     0.692169     0.692191           0.692169 -0.126626
4    2.527693     4.541493     4.541544           4.541493 -0.126626
explained_variance_ratio: [ 0.70227567]
pca: [[ 0.3993854   0.52374084  0.52373976  0.52374084  0.13258598]]
explained_variance_ratio: [ 0.70227567  0.19548756]
pca: [[ 0.3993854   0.52374084  0.52373976  0.52374084  0.13258598]
 [ 0.14836612 -0.11931799 -0.11932505 -0.11931799  0.96709641]]
